In [ ]:
import requests
import json
from datetime import datetime, timedelta
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
import joblib

# Initialize global variables
data_buffer = []

# Fetch data from the API
def fetch_data_from_api():
    url = "https://api.upstox.com/v2/market-quote/ohlc"
    headers = {"Accept": "application/json"}

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        print("Failed to fetch data from the API:", response.status_code)
        return None

# Process data, update buffer, and train model
def process_data_and_train_model(json_data, model):
    if json_data is None:
        return

    ohlc_data = json_data['data']['NSE_EQ:NHPC']['ohlc']
    timestamp = json_data['data']['NSE_EQ:NHPC']['timestamp']

    # Extract OHLC data and timestamp
    open_price = ohlc_data['open']
    high_price = ohlc_data['high']
    low_price = ohlc_data['low']
    close_price = ohlc_data['close']

    # Append new data to buffer
    data_buffer.append((timestamp, open_price, high_price, low_price, close_price))

    # Remove old data from buffer (keep only last 60 minutes data)
    current_time = datetime.fromisoformat(timestamp.split('+')[0])
    cutoff_time = current_time - timedelta(minutes=60)
    data_buffer[:] = [(t, o, h, l, c) for t, o, h, l, c in data_buffer if t >= cutoff_time]

    # Prepare features and target
    X = np.array([[o, h, l] for _, o, h, l, _ in data_buffer])
    y = np.array([c for _, _, _, _, c in data_buffer])

    # Train the model
    model.fit(X, y)

    # Save the trained model
    joblib.dump(model, 'trained_model.pkl')

# Load or initialize the model
def load_or_initialize_model():
    try:
        # Load the model if it exists
        model = joblib.load('trained_model.pkl')
        print("Model loaded successfully.")
    except:
        # Initialize the model if it doesn't exist
        model = RandomForestRegressor(random_state=42)
        print("Model initialized.")

    return model

# Main loop to continuously fetch data, update buffer, train model, predict, and display result
def main():
    model = load_or_initialize_model()

    while True:
        # Fetch data from the API
        json_data = fetch_data_from_api()
        if json_data is None:
            continue

        # Process data, update buffer, and train model
        process_data_and_train_model(json_data, model)

        # Predict OHLC after 10 minutes
        # Here, you can modify the prediction logic based on your requirement

# Run the main function
if __name__ == "__main__":
    main()
